In [1]:
import pandas as pd
import pickle
import yaml
import os
import sys

In [2]:
# Local packages
sys.path.append('../src')
from ninetynine import ninetynine

In [3]:
#open data
with open('../config/paths.yaml','r') as file:
    paths_list = yaml.safe_load(file)
    PATH = os.path.abspath(paths_list['PATH'])

data_imm = pd.read_csv(os.path.join(PATH,'raw','GPP_Final_Output_allTime.csv'))
data_imm.head()

,STUDY_NO,CLASS_CODE,CLASS_DESCRIPTOR,SUBJECT_ID,STUDY_DAY,NAME,ENTRY
0,SARS-COV-2-NHP-064E-1,0,Mock,B03843,B1,G-CSF,0.00
1,SARS-COV-2-NHP-064E-1,0,Mock,B03869,B1,G-CSF,274.61
2,SARS-COV-2-NHP-064E-1,0,Mock,B03928,B1,G-CSF,0.00
3,SARS-COV-2-NHP-064E-1,1,Virus,B03955,B1,G-CSF,136.01
4,SARS-COV-2-NHP-064E-1,1,Virus,B03989,B1,G-CSF,96.51


In [4]:
# Views of the raw data
#data_imm.astype('object').describe()
#len(data_imm[data_imm.NAME == 'C-Reactive Protein'])
#data_imm.loc[data_imm.NAME == 'C-Reactive Protein',['SAMPLE_NUMBER','SUBJECT_ID','STUDY_DAY','STUDY_NO']]
#data_imm.loc[data_imm.NAME == 'ICT',['SAMPLE_NUMBER','SUBJECT_ID','STUDY_DAY','STUDY_NO','ENTRY']].astype(object).describe()
#data_imm.info()
#data_imm.loc[data_imm['SUBJECT_ID'] == 'B03757','NAME']
#data_imm.loc[(data_imm['SUBJECT_ID'] == 'G21E') & (data_imm['NAME'] == 'Hemoglobin')]

### Rename columns and values

In [5]:
# Rename columns and values to follow previous convention
data_imm.rename(columns = {'SUBJECT_ID':'Subject','CLASS_DESCRIPTOR':'Class','STUDY_DAY':'StudyDay'},inplace=True)
data_imm.loc[data_imm.Class == 'Virus','Class'] = 'Infected'

# Get rid of spaces and slashes, these are annoying later
data_imm['NAME'] = data_imm['NAME'].replace(' ','_',regex=True)
data_imm['NAME'] = data_imm['NAME'].replace('/','v',regex=True)

### Reshape table

In [6]:
id_cols_imm = ['STUDY_NO','Class','Subject','StudyDay']

In [7]:
# Check that the pivot won't be averaging any rows together by outputting the count instead of the mean (All values should be 1)
test = data_imm.pivot_table(values='ENTRY',columns='NAME',index=id_cols_imm,aggfunc=lambda x: len(x.unique()))
var_cols_imm = test.columns.tolist() # save for later
# Check for any values over 1
ninetynine(test[test > 1].any().any(),'merging values across rows')

FALSE: I've got 99 problems, but merging values across rows is not one


In [8]:
# Perform table pivot
data_imm = data_imm.pivot_table(values='ENTRY',columns='NAME',index=id_cols_imm)
data_imm = data_imm.reset_index()
data_imm.head()

NAME,STUDY_NO,Class,Subject,StudyDay,G-CSF,GM-CSF,IFN-Alpha,IFN-gamma,IL-10,IL-12_23_(p40),...,IL-5,IL-6,IL-8,MCP-1,MIP-1alpha,MIP-1beta,TGF-alpha,TNF-alpha,VEGF,sCD40L
0,SARS-COV-2-NHP-064E-1,Infected,B03757,-11,NaN,NaN,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SARS-COV-2-NHP-064E-1,Infected,B03757,10,9141.11,205.31,17.24,2759.65,27508.34,10434.72,...,273.10,872.68,493.04,437.84,293.13,302.13,605.24,5153.26,7274.82,10984.0
2,SARS-COV-2-NHP-064E-1,Infected,B03757,12,13507.93,353.40,14.67,4484.62,44871.80,10000.00,...,340.96,1036.62,667.74,430.18,423.91,383.60,440.00,8209.15,12822.64,1984.0
3,SARS-COV-2-NHP-064E-1,Infected,B03757,19,11646.09,120.76,0.00,4342.25,NaN,10000.00,...,118.00,627.93,812.40,385.25,412.36,405.16,440.00,3535.28,NaN,63603.3
4,SARS-COV-2-NHP-064E-1,Infected,B03757,2,103.98,18.27,410.82,44.80,218.58,30.34,...,0.00,47.84,140.13,629.07,36.33,153.07,5.30,24.28,7.28,604.5


In [9]:
# check for duplicates after the table reshaping
ninetynine(len(data_imm[data_imm.duplicated(subset=['Subject','StudyDay'])]) > 0, 'duplicate values')

FALSE: I've got 99 problems, but duplicate values is not one


### Calculate delta terms

In [10]:
# Create temporary dataframe to add the delta timepoints
data_imm = data_imm.set_index(['Subject','StudyDay']).unstack()

for var in var_cols_imm:
    # Collapse the pre-exposure values (should have one per subject)
    data_imm.loc[:,(var,'pre')] = data_imm.loc[:,(var,['-11','-7','B1'])].mean(axis = 1, skipna = True)
    test = data_imm.loc[:,(var,['-11','-7','B1'])].count(axis = 1)
    # Calculate the change from pre-exposure
    data_imm.loc[:,(var,'pre_delta')] = data_imm.loc[:,(var,'pre')]-data_imm.loc[:,(var,'pre')] #should be all 0
    data_imm.loc[:,(var,'2_delta')] = data_imm.loc[:,(var,'2')]-data_imm.loc[:,(var,'pre')]
    data_imm.loc[:,(var,'4_delta')] = data_imm.loc[:,(var,'4')]-data_imm.loc[:,(var,'pre')]
    data_imm.loc[:,(var,'6_delta')] = data_imm.loc[:,(var,'6')]-data_imm.loc[:,(var,'pre')]
    data_imm.loc[:,(var,'8_delta')] = data_imm.loc[:,(var,'8')]-data_imm.loc[:,(var,'pre')]

In [11]:
# Check for any values over 1
ninetynine(test[test > 1].any().any(),'multiple baseline values')

FALSE: I've got 99 problems, but multiple baseline values is not one


In [12]:
# stack and unstack so that missing columns fill in with null
data_imm = data_imm.stack().unstack()

# Copy class to other time points
time_points=data_imm.loc[:,'Class'].columns.to_list()
# back-fill and foward-fill incase the column order changes
data_imm.loc[:,('Class',time_points)] = data_imm.loc[:,('Class',time_points)].fillna(method='ffill',axis=1).fillna(method='bfill',axis=1)

In [13]:
# Reformat for readability
data_imm = data_imm.stack()

### Useful views of the data

In [14]:
# Useful views of the data for troubleshooting error values

#id_cols_imm = ['SAMPLE_NUMBER','STUDY_NO','CLASS_CODE','Class','Subject','SEX','DOB','SPECIES_STRAIN','COUNTRY_ORIGIN','StudyDay','SAMPLED_DATE'
#data_imm.iloc[5354]
#data_imm.NAME.unique()
#data_imm.loc[(data_imm.NAME == 'C-Reactive Protein') & (data_imm.ENTRY == '<0.1'),'ENTRY'] = 0
#data_imm.loc[data_imm.NAME == 'Creatinine Kinase','ENTRY'].unique()
#data_imm.describe()
#data_imm.loc[data_imm.Subject == 'B03757',['STUDY_NO','CLASS_CODE','Class','Subject','SEX','DOB','SPECIES_STRAIN','COUNTRY_ORIGIN','StudyDay','SAMPLED_DATE','ANALYSIS']]
#data_imm.loc[data_imm.Subject == 'H54R',id_cols_imm]
#data_imm.StudyDay.unique()

#len(data_imm.loc[(data_imm.Subject == 'B03757') & (data_imm.ANALYSIS == 'CHEM_CATALYSTONE')].dropna(how='all',axis=1).columns)
#len(data_imm.loc[(data_imm.Subject == 'B03757') & (data_imm.ANALYSIS == 'CBC_PROCYTEDX')].dropna(how='all',axis=1).columns)
#len(data_imm.loc[data_imm.Subject == 'B03757'].dropna(how='all',axis=1).columns)
#len(data_imm.loc[data_imm.Subject == 'B03757'].columns)
#all_col = data_imm.columns.to_list().remove(id_cols_imm)
#all_sanNA = data_imm.loc[data_imm.Subject == 'B03757'].dropna(how='all',axis=1).columns

#data_imm.describe().loc['count',:]
#data_imm.info()

data_imm.groupby(['StudyDay','Class']).agg(['count','mean','std','min','max'])

#data_imm.describe()
#data_imm.loc[data_imm.Subject == 'H54R']

#data_imm.loc[(data_imm.Class == 'Mock') & (data_imm.StudyDay == '2')]

G-CSF                                             GM-CSF  \
                   count         mean          std     min       max  count   
StudyDay  Class                                                               
-11       Infected     0          NaN          NaN     NaN       NaN      0   
          Mock         0          NaN          NaN     NaN       NaN      0   
-7        Infected     0          NaN          NaN     NaN       NaN      0   
          Mock         0          NaN          NaN     NaN       NaN      0   
10        Infected     3  3143.806667  5195.845016    0.00   9141.11      3   
          Mock         3    99.993333   144.702332    0.00    265.92      3   
12        Infected     3  4611.273333  7705.332539   66.64  13507.93      3   
          Mock         3   101.363333   132.810348   13.44    254.14      3   
19        Infected     3  3975.343333  6643.634032   52.75  11646.09      3   
          Mock         3    83.470000   144.574281    0.00    250.41      3   
2         Infected    10    32.499000    39.673494    0.00    103.98     12   
          Mock        12    38.949167    64.297845    0.00    217.25     12   
2_delta   Infected     9    15.478889    33.746866   -1.97    103.98     12   
          Mock        12    14.146667    38.285263  -57.36    110.69     12   
30        Infected     3  2056.093333  3336.693375  100.78   5908.83      3   
          Mock         3   117.253333   122.421740    0.00    244.26      3   
4         Infected    12    14.416667    33.883916    0.00    118.66     12   
          Mock        12    32.560833    67.322123    0.00    234.03     12   
4_delta   Infected    11   -13.652727    64.013881 -136.01    118.66     12   
          Mock        12     7.758333    28.172440  -40.58     76.81     12   
6         Infected    10    28.241000    48.518492    0.00    146.76     12   
          Mock        12    24.851667    50.202135    0.00    179.22     12   
6_delta   Infected     9     9.922222    53.892323  -39.57    146.76     12   
          Mock        12     0.049167    33.761134  -95.39     34.06     12   
8         Infected     0          NaN          NaN     NaN       NaN      0   
          Mock         0          NaN          NaN     NaN       NaN      0   
8_delta   Infected     0          NaN          NaN     NaN       NaN      0   
          Mock         0          NaN          NaN     NaN       NaN      0   
B1        Infected    11    29.380000    46.080673    0.00    136.01     12   
          Mock        12    24.802500    78.903645    0.00    274.61     12   
NEC       Infected     9    17.325556    17.306561    0.32     59.01      9   
          Mock         9    16.487778    16.183439    0.00     50.03      9   
pre       Infected    11    29.380000    46.080673    0.00    136.01     12   
          Mock        12    24.802500    78.903645    0.00    274.61     12   
pre_delta Infected    11     0.000000     0.000000    0.00      0.00     12   
          Mock        12     0.000000     0.000000    0.00      0.00     12   

                                                           ...  VEGF  \
                          mean         std    min     max  ... count   
StudyDay  Class                                            ...         
-11       Infected         NaN         NaN    NaN     NaN  ...     0   
          Mock             NaN         NaN    NaN     NaN  ...     0   
-7        Infected         NaN         NaN    NaN     NaN  ...     0   
          Mock             NaN         NaN    NaN     NaN  ...     0   
10        Infected   75.440000  112.500422   7.92  205.31  ...     3   
          Mock       15.096667    3.898850  11.50   19.24  ...     3   
12        Infected  128.203333  195.061605  11.88  353.40  ...     3   
          Mock       19.170000    5.402194  15.29   25.34  ...     3   
19        Infected   50.920000   60.670230  11.24  120.76  ...     2   
          Mock       11.600000    7.931715   2.91   18.45  ...     3   
2

In [15]:
# Save full table (no error values)
data_imm.to_csv(os.path.join(PATH,'tables','data_imm.csv'))

# Save simplified table (only delta timepoints)
data_imm = data_imm.reset_index()
data_imm_delta = data_imm.drop(data_imm[data_imm['StudyDay'].isin(['10','12','19','30','B1','NEC','-11','-7','2','4','6','8','pre','pre_delta'])].index)
data_imm_delta.set_index(['Subject','StudyDay']).to_csv(os.path.join(PATH,'tables','data_imm_delta.csv'))

In [16]:
print('-- Count of NA entries for each column --')
for var in var_cols_imm:
    na_count = data_imm_delta.loc[data_imm_delta.StudyDay.isin(['2_delta','4_delta','6_delta']),var].isna().sum()
    if na_count > 0:
        ninetynine(na_count > 0,'Missing values ('+var+' - '+str(na_count)+')')

-- Count of NA entries for each column --
TRUE: I've got 99 problems, and Missing values (G-CSF - 7) is the 100th
TRUE: I've got 99 problems, and Missing values (TGF-alpha - 1) is the 100th


In [17]:
# Remove variables with missing values
var_cols_imm.remove('G-CSF')
var_cols_imm.remove('TGF-alpha')

In [18]:
var_cols_imm

['GM-CSF',
 'IFN-Alpha',
 'IFN-gamma',
 'IL-10',
 'IL-12_23_(p40)',
 'IL-13',
 'IL-15',
 'IL-17a',
 'IL-18',
 'IL-1b',
 'IL-1ra',
 'IL-2',
 'IL-4',
 'IL-5',
 'IL-6',
 'IL-8',
 'MCP-1',
 'MIP-1alpha',
 'MIP-1beta',
 'TNF-alpha',
 'VEGF',
 'sCD40L']

In [19]:
# save lists
with open('../config/lists_imm.pkl', 'wb') as f:
    pickle.dump([id_cols_imm,var_cols_imm], f)

In [20]:
data_imm_delta

NAME,Subject,StudyDay,Class,G-CSF,GM-CSF,IFN-Alpha,IFN-gamma,IL-10,IL-12_23_(p40),IL-13,...,IL-6,IL-8,MCP-1,MIP-1alpha,MIP-1beta,STUDY_NO,TGF-alpha,TNF-alpha,VEGF,sCD40L
6,B03757,2_delta,Infected,103.98,7.28,410.82,31.13,-39.18,-11.57,-25.55,...,29.19,124.26,510.64,36.33,22.94,NaN,1.84,2.14,7.28,604.50
9,B03757,4_delta,Infected,118.66,9.07,0.00,10.56,119.17,10.76,4.97,...,24.92,165.65,393.94,33.04,18.52,NaN,10.47,18.12,3.46,689.80
11,B03757,6_delta,Infected,146.76,12.76,0.00,422.08,5422.86,1397.31,269.20,...,187.17,90.74,242.25,53.29,10.73,NaN,78.16,421.31,280.70,528.85
13,B03757,8_delta,Infected,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,B03781,2_delta,Infected,-1.97,0.51,207.62,2.69,8.03,0.13,3.85,...,-0.08,67.59,103.44,2.00,2.20,NaN,0.00,15.23,-0.33,2.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,H56N,8_delta,Mock,NaN,NaN,0.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
420,H89K,2_delta,Mock,2.56,0.14,0.00,-1.16,8.30,-1.92,0.59,...,3.04,4.81,40.67,-1.15,0.48,NaN,-0.10,0.00,17.43,38.90
423,H89K,4_delta,Mock,17.67,0.58,0.00,0.95,4.83,1.51,6.85,...,1.77,45.54,45.90,0.00,0.66,NaN,0.06,14.93,0.00,90.47
425,H89K,6_delta,Mock,3.35,0.17,0.00,-0.81,8.53,-2.31,-0.62,...,1.19,6.05,17.11,-4.58,0.91,NaN,-0.13,11.59,0.00,17.16
